In [12]:
import json
import tensorflow as tf

def load_data(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)

    image_paths = []
    keypoints = []
    for item in data:
        image_paths.append("data/images/" + str(item['id']) + '.png')  # Assuming JPG format
        keypoints.append(item['kps'])  # List of 28 keypoints (x, y)

    return image_paths, keypoints

train_image_paths, train_keypoints = load_data('data/data_train.json')
val_image_paths, val_keypoints = load_data('data/data_val.json')

train_image_paths = train_image_paths + val_image_paths
train_keypoints = train_keypoints + val_keypoints


In [13]:
def parse_function(filename, keypoints):
    image = tf.io.read_file(filename)
    image = tf.image.decode_image(image, channels=3)  # Decode the image
    # image = tf.image.convert_image_dtype(image, tf.float32)  # Convert to float32

    # Ensure the image tensor has a known shape
    image = tf.ensure_shape(image, [None, None, 3])

    # Resize the image
    image = tf.image.resize(image, (224, 224))  # Resize image

    keypoints  = tf.reshape(keypoints, [-1, 2])
    keypoints *= tf.constant([[224/1280, 224/720]])
    keypoints  = tf.reshape(keypoints, [-1])

    return image, keypoints


def create_dataset(image_paths, keypoints):
    image_paths = tf.constant(image_paths)
    keypoints = tf.constant(keypoints, dtype=tf.float32)
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, keypoints))
    dataset = dataset.map(parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    return dataset

train_dataset = create_dataset(train_image_paths, train_keypoints)

In [14]:
def prepare_for_training(ds, batch_size=128, shuffle_buffer_size=10000):  
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds

train_dataset = prepare_for_training(train_dataset)

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping

def create_model():
    ## Define input layer
    inputs = Input(shape=(224, 224, 3))
    preprocessed_input = preprocess_input(inputs)  # Preprocess input images
    
    ## Load pre-trained model “Resnet50” without the final(top) layer
    base_model = MobileNetV3Small(weights='imagenet', include_top=False, input_tensor=preprocessed_input)
    base_model.trainable = False
    output = base_model.output

    ## Condense feature maps from the output
    output = Flatten()(output)


    # Final layer has 28 output neurons
    final_output = Dense(28, activation='relu')(output)  

    ## Create our own network/model
    model = Model(inputs=inputs, outputs=final_output)

    return model


model = create_model()
model.compile(optimizer=tf.keras.optimizers.Adam(.0001), loss='mae')  # Using mean squared error loss for regression task

earlystopping = EarlyStopping(monitor="loss", patience=5, restore_best_weights=True)


In [16]:
history = model.fit(train_dataset, epochs=200, callbacks=[earlystopping])

Epoch 1/200
70/70 [==============================] - 31s 111ms/step - loss: 64.3953
Epoch 2/200
70/70 [==============================] - 29s 108ms/step - loss: 12.2406
Epoch 3/200
70/70 [==============================] - 26s 106ms/step - loss: 6.6135
Epoch 4/200
70/70 [==============================] - 25s 106ms/step - loss: 5.4773
Epoch 5/200
70/70 [==============================] - 25s 110ms/step - loss: 4.7634
Epoch 6/200
70/70 [==============================] - 26s 109ms/step - loss: 4.2595
Epoch 7/200
70/70 [==============================] - 26s 109ms/step - loss: 3.8679
Epoch 8/200
70/70 [==============================] - 26s 107ms/step - loss: 3.5608
Epoch 9/200
70/70 [==============================] - 25s 108ms/step - loss: 3.3026
Epoch 10/200
70/70 [==============================] - 26s 112ms/step - loss: 3.0860
Epoch 11/200
70/70 [==============================] - 26s 111ms/step - loss: 2.8903
Epoch 12/200
70/70 [==============================] - 29s 109ms/step - loss: 2.7292

In [22]:
model.save("models/keypoints_ 0_3093loos.model.h5")

In [20]:
import pickle

# Save the history variable to a file
with open('training_history.pkl', 'wb') as file:
    pickle.dump(history.history, file)
